In [31]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
import torchvision.transforms as transforms

from torchvision import datasets
from torch.utils.data import DataLoader
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.manifold import MDS

In [32]:
alexnet = models.alexnet(pretrained=True)
alexnet = torch.nn.Sequential(*(list(alexnet.features.children())[:-1]))
alexnet.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load data
data_folder = 'Images'
image_files = sorted(os.listdir(data_folder))

In [37]:
# Assign arbitrary class labels
class_labels = ['Animals']*28 + ['Objects']*36 + ['Scenes']*36 + ['Human activities']*24 + ['Faces']*21

# Create dataset
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform

    def __getitem__(self, index):
        image_path = os.path.join(self.image_folder, image_files[index])
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = class_labels[index]
        return image, label

    def __len__(self):
        return len(image_files)

data = CustomImageDataset(data_folder, transform=transform)
data_loader = DataLoader(data, batch_size=1, shuffle=False)

# Function to extract activations
def extract_activations(model, dataloader):
    activations = {}
    for layer_name, layer in model.named_children():
        activations[layer_name] = []
    with torch.no_grad():
        for inputs, _ in dataloader:
            features = inputs
            for layer_name, layer in model.named_children():
                features = layer(features)
                if isinstance(layer, torch.nn.modules.conv.Conv2d) or \
                   isinstance(layer, torch.nn.modules.linear.Linear):
                    activations[layer_name].append(features.cpu().numpy().flatten())
    for key in activations:
        activations[key] = np.array(activations[key])
    return activations

# Extract activations
activations = extract_activations(alexnet, data_loader)

IndexError: list index out of range